<a href="https://colab.research.google.com/github/BetriuJaume/TFG-code/blob/main/Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code for the deep learning model:

In [ ]:
!pip install pyreadstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 5.0 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.utils.data as data_utils
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import warnings

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import resample

In [ ]:
# Reproducibility:
torch.manual_seed(0)

In [ ]:
dades_orig=pd.read_spss('EarlyLifeCovidPACIENTES V6 ID.sav')
dades_extra=pd.read_spss('extra_data.sav')

dades_extra['Talla_mt'] = dades_extra['Talla']/100
dades_extra=dades_extra.rename(columns = {'Tabaco_Sino':'Tabaco_SIno'})
dades_extra=dades_extra.drop(list(set(dades_extra.columns)-set(dades_orig)),axis=1)
dades_extra['IUGR_missing']= [1.]*len(dades_extra)

for colu in dades_extra.columns[12:-2]:
    dades_extra[colu]=['si' if x==1 else 'no' for x in dades_extra[colu]]

dades=pd.concat([dades_orig, dades_extra], ignore_index=True)

def canvi_tab(elem):
  if((elem=='si (pasado o actual)') | 
     (elem=='si (actual o pasado)')):
    return 'yes'
  if(elem=='nunca'):
    return 'no'
def canvi(elem):
  if(elem=='si'):
    return 'yes'
  if(elem=='no'):
    return 'no'
def canvi_UCI(elem):
  if(elem==1):
    return 'Yes'
  if(elem==0):
    return 'No'
def canvi_sex(elem):
  if(elem=='mujer'):
    return 'woman'
  if(elem=='hombre'):
    return 'man'

dades['Tabaco_SIno']=dades['Tabaco_SIno'].apply(canvi_tab)
dades['UCI']=dades['UCI'].apply(canvi_UCI)
for columna in dades.columns[14:]:
  dades[columna]=dades[columna].apply(canvi)
dades['Gender']=dades['Gender'].apply(canvi_sex)
dades['IUGR_calc']=dades['IUGR_calc'].apply(canvi_UCI)

dades_no_translate = dades.copy()

dades.columns=pd.Index(['ID', 'UCI', 'IUGR_missing', 'Age', 'Gender', 'Size', 'Weight',
       'Size_mt', 'IMC', 'BW', 'BW_2500', 'percentil_birth', 'IUGR_calc',
       'Tobacco_yes_no', 'Hipertension', 'Heart_diseases', 'DM', 'Dyslipidemia',
       'Obesity', 'Kidney_disease', 'Autoimmune', 'Cancer', 'Thyroid', 'Infectious',
       'Psychiatric'])

dades['Age']=dades['Age'].fillna(dades['Age'].mean())
dades['Size']=dades['Size'].fillna(dades['Size'].mean())
dades['Weight']=dades['Weight'].fillna(dades['Weight'].mean())
dades['Size_mt']=dades['Size']/100
dades['IMC']=dades['IMC'].fillna(dades['IMC'].mean())
dades['percentil_birth']=dades['percentil_birth'].fillna(dades['percentil_birth'].mean())
dades['IUGR_calc']=dades['IUGR_calc'].fillna(dades['IUGR_calc'].mode().iloc[0])

In [ ]:
dades_train = dades.sample(frac=0.8, random_state=25).drop('ID',axis=1)
dades_test = dades.drop(dades_train.index).drop('ID',axis=1)

# Design, training and hiperparameter adjusting of the Neural Network

## Preprocessing of the data:

Codification of the data into Torch tensors:

In [ ]:
#Functions that will help for the codification:

def codGender(elem):
  if elem=='man':
    return 1.
  else:
    return 0.
def codUCI(elem):
  if elem=='Yes':
    return 1.
  else:
    return 0.
def codBW(elem):
  if elem=='BW normal':
    return 0.
  else:
    return 1.
def cod(elem):
  if elem=='yes':
    return 1.
  else:
    return 0.

In [ ]:
dades_cod_train=dades_train.drop(['IUGR_missing', 'Size', 'BW'],axis=1).copy()
dades_cod_test=dades_test.drop(['IUGR_missing', 'Size', 'BW'],axis=1).copy()

for df in [dades_cod_train,dades_cod_test]:
  df['UCI']=df['UCI'].apply(codUCI)
  df['Gender']=df['Gender'].apply(codGender)
  df['BW_2500']=df['BW_2500'].apply(codBW)
  df['IUGR_calc']=df['IUGR_calc'].apply(codUCI)
  for element in df.columns[9:]:
    df[element]=df[element].apply(cod)

target_train=torch.tensor(dades_cod_train['UCI'].values)
features_train=torch.tensor(dades_cod_train.drop('UCI',axis=1).values)
Dades_Tensor_train=TensorDataset(features_train,target_train)

target_test=torch.tensor(dades_cod_test['UCI'].values)
features_test=torch.tensor(dades_cod_test.drop('UCI',axis=1).values)
Dades_Tensor_test=TensorDataset(features_test,target_test)

In [ ]:
train_torch = Dades_Tensor_train
test_torch = Dades_Tensor_test

train_torch1=DataLoader(train_torch,batch_size=100,shuffle=True)
test_torch1=DataLoader(test_torch,batch_size=len(features_test),shuffle=True)

check if CPU acceleration is activated:

In [ ]:
if torch.cuda.is_available():
  print('Cuda available')
else:
  print('Cuda not available')

Cuda available


## Design and training:
We begin building the class for the network:

In [ ]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.input_layer=nn.Linear(features_train.size()[1],14) #weights
    #self.hidden1=nn.Linear(14,14)
    #self.hidden2=nn.Linear(7,7)
    #self.hidden3=nn.Linear(7,10)
    #self.hidden4=nn.Linear(10,7)
    self.output=nn.Linear(14,2)
  
  def forward(self, data): #activation functions, we will use the ReLu function
    data=F.relu(self.input_layer(data))
    #data=F.relu(self.hidden1(data))
    #data=F.relu(self.hidden2(data))
    #data=F.relu(self.hidden3(data))
    #data=F.relu(self.hidden4(data))
    data=self.output(data)
    return F.softmax(data, dim=1)

Train and test functions:

In [ ]:
def train_one_epoch(model, data_loader, loss_function, optimiser, device):
  for inputs, targets in data_loader:
    inputs, targets = inputs.to(device), targets.to(device)

    # We have to calculate the loss
    predictions = model(inputs.float())
    loss = loss_function(predictions, targets.long())

    # Update the weights
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

  print(f"Loss: {loss.item()}")


def train(model, data_loader, loss_function, optimiser, device, epochs):
  for i in range(epochs):
    print(f"Epoch {i}")
    train_one_epoch(model, data_loader, loss_function, optimiser, device)
    print('_______________________')
   
  print('End of training')



def predict(model, input, target):
  model.eval() #we activate eval so the network knows we are testing
  with torch.no_grad():
    predictions = model(input) #.argmax(dim=1)
    return predictions

In [ ]:
network = Network().to("cuda")

#training the network:
train(model=network, 
      data_loader=train_torch1, 
      loss_function=F.nll_loss, 
      device="cuda", 
      optimiser=optim.Adam(network.parameters(),lr=0.0001),
      epochs=10)

#We store the model in case we want to use it in another script:
torch.save(network.state_dict(), "NN_model_for_COVID-19_patients.pth")

Epoch 0
Loss: -0.9230532050132751
_______________________
Epoch 1
Loss: -0.9615135192871094
_______________________
Epoch 2
Loss: -0.9999818205833435
_______________________
Epoch 3
Loss: -0.9999852776527405
_______________________
Epoch 4
Loss: -0.9999797344207764
_______________________
Epoch 5
Loss: -0.9615230560302734
_______________________
Epoch 6
Loss: -0.9615351557731628
_______________________
Epoch 7
Loss: -0.9999963045120239
_______________________
Epoch 8
Loss: -0.9999987483024597
_______________________
Epoch 9
Loss: -0.8846145868301392
_______________________
End of training


We can see that the model has a very bad performance even in the train data. This is shown by the values of the log likelihood cost function. This might we caused because the network has very few examples of patients with a 'Yes' in UCI.

In [ ]:
# Check the results. First we will load the model from "NN_model_for_COVID-19_patients.pth"
network_for_test = Network()
state_dict = torch.load("NN_model_for_COVID-19_patients.pth")
network_for_test.load_state_dict(state_dict)

for input, target in test_torch1:
  predictions = predict(network_for_test, input.float(), target)

print(predictions)

tensor([[1.0000e+00, 5.5092e-18],
        [1.0000e+00, 3.8686e-13],
        [1.0000e+00, 1.2485e-12],
        [1.0000e+00, 2.7915e-11],
        [1.0000e+00, 3.5713e-11],
        [1.0000e+00, 2.5624e-14],
        [1.0000e+00, 7.0726e-22],
        [1.0000e+00, 2.5692e-23],
        [1.0000e+00, 9.2238e-18],
        [1.0000e+00, 1.0203e-17],
        [1.0000e+00, 1.4708e-13],
        [1.0000e+00, 3.7881e-15],
        [1.0000e+00, 5.1226e-14],
        [1.0000e+00, 6.6728e-14],
        [1.0000e+00, 1.4732e-15],
        [1.0000e+00, 6.8183e-20],
        [1.0000e+00, 8.9012e-12],
        [1.0000e+00, 1.7975e-20],
        [1.0000e+00, 2.2305e-16],
        [1.0000e+00, 2.9087e-19],
        [1.0000e+00, 1.2714e-19],
        [1.0000e+00, 4.2847e-17],
        [1.0000e+00, 3.1279e-13],
        [1.0000e+00, 7.2486e-18],
        [1.0000e+00, 4.3481e-11],
        [1.0000e+00, 1.1529e-13],
        [1.0000e+00, 3.8934e-18],
        [1.0000e+00, 2.7604e-19],
        [1.0000e+00, 2.6488e-15],
        [1.000

We can see that the network classifies almost all the patients as the label 0

In [ ]:
print(classification_report(dades_cod_test['UCI'], predictions.argmax(dim=1).tolist()))

              precision    recall  f1-score   support

         0.0       0.95      1.00      0.97       432
         1.0       0.00      0.00      0.00        25

    accuracy                           0.95       457
   macro avg       0.47      0.50      0.49       457
weighted avg       0.89      0.95      0.92       457



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


And has bad results, especially predicting the values for Yes. The strategy then will be to try with the upsampled data to see if we can get the network to work a little bit more on the 'Yes' cases:

In [ ]:
#We take just the 70 percent of the data because we will use a part of the test data to do validation for one hiperparameter
dades_train_70 = dades.sample(frac=0.7, random_state=25).drop('ID',axis=1)
dades_test = dades.drop(dades_train.index).drop('ID',axis=1)

dades_train_UCI_No = dades_train_70[dades_train_70['UCI']=='No']
dades_train_UCI_Yes = dades_train_70[dades_train_70['UCI']=='Yes']
dades_UCI_Yes_upsampled = resample(dades_train_UCI_Yes,replace=True,n_samples=len(dades_train_UCI_No),random_state=123)
dades_train_upsampled = pd.concat([dades_train_UCI_No,dades_UCI_Yes_upsampled])

In [ ]:
dades_cod_train_upsampled = dades_train_upsampled.drop(['IUGR_missing', 'Size', 'BW'],axis=1).copy()
dades_cod_test = dades_test.drop(['IUGR_missing', 'Size', 'BW'],axis=1).copy()

for df in [dades_cod_train_upsampled,dades_cod_test]:
  df['UCI']=df['UCI'].apply(codUCI)
  df['Gender']=df['Gender'].apply(codGender)
  df['BW_2500']=df['BW_2500'].apply(codBW)
  df['IUGR_calc']=df['IUGR_calc'].apply(codUCI)
  for element in df.columns[9:]:
    df[element]=df[element].apply(cod)

target_train_upsampled = torch.tensor(dades_cod_train_upsampled['UCI'].values)
features_train_upsampled = torch.tensor(dades_cod_train_upsampled.drop('UCI',axis=1).values)
Dades_Tensor_train_upsampled = TensorDataset(features_train_upsampled,target_train_upsampled)

target_test = torch.tensor(dades_cod_test['UCI'].values)
features_test = torch.tensor(dades_cod_test.drop('UCI',axis=1).values)
Dades_Tensor_test = TensorDataset(features_test, target_test)

We will create a new datset that we will use to train the decison boundary in the future:

In [ ]:
# We divide into train, validation and test to avoid overfitting:
# train_torch,val_torch=random_split(Dades_Tensor_train,[int(0.8*len(Dades_Tensor_train))+1,int(0.2*len(Dades_Tensor_train))],generator=torch.Generator().manual_seed(123))
train_torch_upsampled = Dades_Tensor_train_upsampled
test_torch, val_torch = random_split(Dades_Tensor_test,[int(0.5*len(Dades_Tensor_test))+1,int(0.5*len(Dades_Tensor_test))],generator=torch.Generator().manual_seed(123))

train_torch1_upsampled = DataLoader(train_torch_upsampled,batch_size=50,shuffle=True)
val_torch1 = DataLoader(val_torch,batch_size=len(val_torch),shuffle=True)
test_torch1 = DataLoader(test_torch,batch_size=len(features_test),shuffle=True)

In [ ]:
network_upsampling = Network().to("cuda")

#training the network:
train(model=network_upsampling, 
      data_loader=train_torch1_upsampled, 
      loss_function=F.nll_loss, 
      device="cuda", 
      optimiser=optim.Adam(network.parameters(),lr=0.0001),
      epochs=10)

#We store the model in case we want to use it in another script:
torch.save(network.state_dict(), "NN_model_for_COVID-19_patients_upsampled.pth")

Epoch 0
Loss: -0.4192551076412201
_______________________
Epoch 1
Loss: -0.40497180819511414
_______________________
Epoch 2
Loss: -0.34472084045410156
_______________________
Epoch 3
Loss: -0.4151182174682617
_______________________
Epoch 4
Loss: -0.39549940824508667
_______________________
Epoch 5
Loss: -0.39517104625701904
_______________________
Epoch 6
Loss: -0.29806965589523315
_______________________
Epoch 7
Loss: -0.40386277437210083
_______________________
Epoch 8
Loss: -0.4387633800506592
_______________________
Epoch 9
Loss: -0.30425772070884705
_______________________
End of training


Here as we added more patients with UCI='Yes' the model performs way better in the train data as we see the loss function decrease significantly. Nevertheless there is the possibility of overfitting.

In [ ]:
network_for_test_upsampled = Network()
state_dict_upsampled = torch.load("NN_model_for_COVID-19_patients_upsampled.pth")
network_for_test_upsampled.load_state_dict(state_dict_upsampled)

for input, target in test_torch1:
  predictions_upsampled = predict(network_for_test_upsampled, input.float(), target)

print(predictions_upsampled.argmax(dim=1))

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


But again the model almost predicts everything to be 0 ('No') so we can't use . 
We will use the validation dataset to tune the decision bondary and choose one that gives us better results (for the last results we have been using a decision boundary of 0.5).

So let's do predictions with the validation dataset:

In [ ]:
for input, target in val_torch1:
  predictions_upsampled_val = predict(network_for_test_upsampled, input.float(), target)

predictions_upsampled_val_df = pd.DataFrame(predictions_upsampled_val.numpy())

In [ ]:
with warnings.catch_warnings():
  warnings.simplefilter("ignore") #We ignore the warnings when we predict the whole datatset to be 'Yes' pr 'No'
  accuracies = []

  for prob in np.arange(0.99, 1, 0.000005):
    predictions_upsampled_val_df['new_predictions'] = (predictions_upsampled_val_df[0] >= prob)
    predictions_upsampled_val_df['new_predictions'] = [0. if element_mask else 1. for element_mask in predictions_upsampled_val_df['new_predictions']]
    report = classification_report([elem.tolist() for _, elem in val_torch1][0], predictions_upsampled_val_df['new_predictions'], output_dict=True)
    accuracies.append({'probability limit': prob, 'precision for No': report['0.0']['precision'], 'precision for Yes': report['1.0']['precision']})

In [ ]:
accuracies_df = pd.DataFrame(accuracies)
best_nn = accuracies_df.loc[accuracies_df['precision for No'] >= accuracies_df['precision for No'].max()]
print(best_nn)

      probability limit  precision for No  precision for Yes
1988            0.99994          0.950226           0.285714


The model with the best results in the validation dataset has a decision boundary of 0.99994. Let's check the predictions with this boundary in the test set:

In [ ]:
for input, target in test_torch1:
  predictions_upsampled_test = predict(network_for_test_upsampled, input.float(), target)

predictions_upsampled_test_df = pd.DataFrame(predictions_upsampled_test.numpy())

prob = 0.99994
predictions_upsampled_test_df['new_predictions'] = (predictions_upsampled_test_df[0] >= prob)
predictions_upsampled_test_df['new_predictions'] = [0. if element_mask else 1. for element_mask in predictions_upsampled_test_df['new_predictions']]
report = classification_report([elem.tolist() for _, elem in test_torch1][0], predictions_upsampled_test_df['new_predictions'])

print(report)

              precision    recall  f1-score   support

         0.0       0.95      0.96      0.95       217
         1.0       0.00      0.00      0.00        12

    accuracy                           0.91       229
   macro avg       0.47      0.48      0.48       229
weighted avg       0.90      0.91      0.90       229



The model coninues to work horrible with unseen data

# Deeper netwok:

Let's try with a deeper network with the upsampled data:

In [ ]:
class Deep_network(nn.Module):
  def __init__(self):
    super().__init__()
    self.input_layer=nn.Linear(features_train.size()[1],7) #weights
    self.hidden1=nn.Linear(7,7)
    self.hidden2=nn.Linear(7,7)
    #self.hidden3=nn.Linear(7,10)
    #self.hidden4=nn.Linear(10,7)
    self.output=nn.Linear(7,2)
  
  def forward(self, data): #activation functions, we will use the ReLu function
    data=F.relu(self.input_layer(data))
    data=F.relu(self.hidden1(data))
    data=F.relu(self.hidden2(data))
    #data=F.relu(self.hidden3(data))
    #data=F.relu(self.hidden4(data))
    data=self.output(data)
    return F.softmax(data, dim=1)

In [ ]:
deep_network = Deep_network().to("cuda")

#training the network:
train(model=deep_network, 
      data_loader=train_torch1_upsampled, 
      loss_function=F.nll_loss, 
      device="cuda", 
      optimiser=optim.Adam(deep_network.parameters(),lr=0.0001),
      epochs=10)

#We store the model in case we want to use it in another script:
torch.save(deep_network.state_dict(), "deep_NN_model_for_COVID-19_patients.pth")

Epoch 0
Loss: -0.5025877356529236
_______________________
Epoch 1
Loss: -0.48559218645095825
_______________________
Epoch 2
Loss: -0.490139365196228
_______________________
Epoch 3
Loss: -0.4780896306037903
_______________________
Epoch 4
Loss: -0.4789683222770691
_______________________
Epoch 5
Loss: -0.5023975372314453
_______________________
Epoch 6
Loss: -0.5383976101875305
_______________________
Epoch 7
Loss: -0.5543627738952637
_______________________
Epoch 8
Loss: -0.519188642501831
_______________________
Epoch 9
Loss: -0.5703024864196777
_______________________
End of training


In [ ]:
deep_network_for_test = Deep_network()
state_dict_upsampled = torch.load("deep_NN_model_for_COVID-19_patients.pth")
deep_network_for_test.load_state_dict(state_dict_upsampled)

for input, target in test_torch1:
  predictions_upsampled = predict(deep_network_for_test, input.float(), target)

print(predictions_upsampled.argmax(dim=1))

tensor([0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1,
        1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0,
        0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
        0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1,
        0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
        0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0,
        0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0,
        1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0])


In [ ]:
for input, target in val_torch1:
  predictions_upsampled_val = predict(deep_network_for_test, input.float(), target)

predictions_upsampled_val_df = pd.DataFrame(predictions_upsampled_val.numpy())

In [ ]:
with warnings.catch_warnings():
  warnings.simplefilter("ignore") #We ignore the warnings when we predict the whole datatset to be 'Yes' pr 'No'
  accuracies = []

  for prob in np.arange(0, 1, 0.025):
    predictions_upsampled_val_df['new_predictions'] = (predictions_upsampled_val_df[0] >= prob)
    predictions_upsampled_val_df['new_predictions'] = [0. if element_mask else 1. for element_mask in predictions_upsampled_val_df['new_predictions']]
    report = classification_report([elem.tolist() for _, elem in val_torch1][0], predictions_upsampled_val_df['new_predictions'], output_dict=True)
    accuracies.append({'probability limit': prob, 'precision for No': report['0.0']['precision'], 'precision for Yes': report['1.0']['precision']})

We let the accuracy for the 'No' be a little worst so we can check more models

In [ ]:
accuracies_df = pd.DataFrame(accuracies)
best_nn = accuracies_df.loc[accuracies_df['precision for No'] >= accuracies_df['precision for No'].max() - 0.04]
print(best_nn)

    probability limit  precision for No  precision for Yes
0               0.000          0.942982           0.000000
1               0.025          0.942731           0.000000
2               0.050          0.942478           0.000000
3               0.075          0.946429           0.250000
4               0.100          0.941176           0.000000
5               0.125          0.940092           0.000000
6               0.150          0.949074           0.166667
7               0.175          0.942857           0.055556
8               0.200          0.946602           0.090909
9               0.225          0.939698           0.034483
10              0.250          0.943005           0.057143
11              0.275          0.940217           0.045455
12              0.300          0.938547           0.040816
13              0.325          0.934524           0.033333
14              0.350          0.950000           0.073529
15              0.375          0.935065           0.0405

We will choose prob = 0.075

In [ ]:
prob = 0.075

for input, target in test_torch1:
  predictions_upsampled_test = predict(deep_network_for_test, input.float(), target)

predictions_upsampled_test_df = pd.DataFrame(predictions_upsampled_test.numpy())

predictions_upsampled_test_df['new_predictions'] = (predictions_upsampled_test_df[0] >= prob)
predictions_upsampled_test_df['new_predictions'] = [0. if element_mask else 1. for element_mask in predictions_upsampled_test_df['new_predictions']]
report = classification_report([elem.tolist() for _, elem in test_torch1][0], predictions_upsampled_test_df['new_predictions'])

print(report)

              precision    recall  f1-score   support

         0.0       0.95      0.99      0.97       217
         1.0       0.00      0.00      0.00        12

    accuracy                           0.93       229
   macro avg       0.47      0.49      0.48       229
weighted avg       0.90      0.93      0.92       229



We fail again to get the network to predict focus too on the patiens with UCI = 1